In [1]:
import pandas as pd
import glob
import os
import xml.etree.ElementTree as ET

# Load Data

In [8]:
# Full bvb export

path = 'data/csv/b3kat_export/'
files = glob.glob(os.path.join(path, '*.csv'))

li = []

for filename in files: 
    frame = pd.read_csv(filename, header=0)
    li.append(frame)

df_full = pd.concat(li, axis=0, ignore_index=True)

# No need for all the columns

df_full= df_full[['id', 'lang', 'authors', 'parentId']]

C:\Users\hanna\AppData\Local\Temp/ipykernel_9664/2627611449.py:9: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(filename, header=0)


In [9]:
# signatures

df_sig = pd.read_csv('data/csv/signatures.csv')

df_sig = df_sig[['sys', 'text', 'text_3', 'text_2', 'text_1']]

In [10]:
# Books available in the library

df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0, 
                          names = ['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr', 'inventory_date'])

C:\Users\hanna\AppData\Local\Temp/ipykernel_9664/3202955160.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_freihand = pd.read_csv('data/Freihand.csv', sep=';',  error_bad_lines=False, header=0,


In [11]:
print(df_freihand.columns)
print(df_full.columns)
print(df_sig.columns)

Index(['bvb', 'title', 'year', 'signature', 'collection', 'inventory_nr',
       'inventory_date'],
      dtype='object')
Index(['id', 'lang', 'authors', 'parentId'], dtype='object')
Index(['sys', 'text', 'text_3', 'text_2', 'text_1'], dtype='object')


### Merging Signatures and Books

In [17]:
# Extracting the known signature types

df_freihand[['signature_l', 'signature_n']] = df_freihand.signature.str.extract(r'^([a-zA-Z]{2}) (\d+)', expand=False)


df_freihand['signature_clean'] = df_freihand[['signature_l', 'signature_n']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Merging on the signatures

merged = df_freihand.merge(df_sig, left_on='signature_clean', right_on='sys', how='outer', indicator=True)

# Mergin on the bvb signature

merged = merged.merge(df_full, left_on='bvb', right_on='id')

In [24]:
right_only = merged.loc[merged['_merge'] == 'right_only']

merged._merge.unique()

['left_only', 'both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [16]:
rec_sigs = len(merged) / len(df_freihand) * 100

print(f'Books in Freihand with recognizable signature: {rec_sigs:.1f} %')

Books in Freihand with recognizable signature: 19.7 %


In [55]:
# Should I be doing this?

merged.drop_duplicates(subset='bvb', inplace=True)



In [56]:
merged.columns

Index(['bvb', 'title_x', 'year_x', 'signature', 'collection', 'inventory_nr',
       'inventory_date', 'signature_l', 'signature_n', 'signature_clean',
       'Unnamed: 0', 'lev', 'sys', 'numbis', 'text', 'vw', 'backreference',
       'text_3', 'text_2', 'text_1', 'id', 'lang', 'authors', 'title_y',
       'subtitle', 'statement', 'place', 'publisher', 'year_y', 'editors',
       'subjects', 'subject_types', 'parentId', 'parentTitle'],
      dtype='object')

In [59]:
len(merged)

58640

# Creating one big subject column for testing

In [81]:
merged['subject_list'] = merged[['text','text_3', 'text_2', 'text_1']].apply(lambda x: ' '.join(x.values.astype(str)), axis=1)


In [82]:
merged

,bvb,title_x,year_x,signature,collection,inventory_nr,inventory_date,signature_l,signature_n,signature_clean,...,statement,place,publisher,year_y,editors,subjects,subject_types,parentId,parentTitle,subject_list
0,BV001953292,Sozialgeschichte der mittelalterlichen Kunst,1957,Mk 1020-5571,BIB,2019.4415,20190926,Mk,1020,Mk 1020,...,Arnold Hauser,Hamburg,Rowohlt,1957,NaN,Kunst,s,BV000001769,rororo,interkontinental und Europa allgemein. Mittela...
1,BV038967812,Homo caelestis,1965,Mk 1020-5651/1,BIB,NaN,0,Mk,1020,Mk 1020,...,Wolfram von den Steinen,Bern ; München,Francke,1965,NaN,NaN,NaN,NaN,NaN,interkontinental und Europa allgemein. Mittela...
2,BV037569766,Architektur und monumentale Darstellung im hoh...,1938,Mk 1020-5383,BIB,2019.4925,20191011,Mk,1020,Mk 1020,...,Wolfgang Rothkirch,Leipzig,Seemann,1938,NaN,Architektur,s,NaN,NaN,interkontinental und Europa allgemein. Mittela...
3,BV038914945,Homo caelestis,1965,Mk 1020-5651/2,BIB,NaN,0,Mk,1020,Mk 1020,...,Wolfram von den Steinen,Bern ; München,Francke,1965,NaN,Ikonographie|Jesus Christus|Gleichnis vom Fest...,s|p|u,NaN,NaN,interkontinental und Europa allgemein. Mittela...
4,BV001917937,Das Europa der Kathedralen 1140-1280,1966,Mk 1020-5661,BIB,NaN,0,Mk,1020,Mk 1020,...,Georges Duby ; aus dem Französischen von Karl ...,Genève,Skira,[1966],"Hemmerich, Karl Georg",Europa|Kunst,g|s,NaN,NaN,interkontinental und Europa allgemein. Mittela...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64356,BV012178543,Die Kunstdenkmäler von Bayern,1895,Ua 7110/4 gr,BIB,NaN,0,Ua,7110,Ua 7110,...,im Auftr. des Staatsministeriums für Unterrich...,München,Verl. der Vereinigten Kunstanst. vormals Albert,1895,NaN,NaN,NaN,NaN,NaN,I: Die Kunstdenkmale des Regierungsbezirks Obe...
64357,BV014564974,Die Kunstdenkmäler von Bayern,1905,Ua 7110/5 gr,BIB,NaN,0,Ua,7110,Ua 7110,...,im Auftr. des Staatsministeriums für Unterrich...,München,Verl. der Vereinigten Kunstanst. vormals Albert,1905,NaN,NaN,NaN,NaN,NaN,I: Die Kunstdenkmale des Regierungsbezirks Obe...
64358,BV001109543,Kirkebygninger og deres udstyr,1934,St 160-5330,BIB,NaN,0,St,160,St 160,...,utg. av Vilh. Lorenzen,Stockholm,Bonnier [u.a.],1934,"Lorenzen, Vilhelm",Kirchenbau|Nordische Staaten,s|g,BV000892704,Nordisk kultur,nordeuropäische Länder Kirchenausstattung allg...
64359,BV048495529,Dressed for immortality,2022,Sv 1200-6220,BIB,2022.4675,20221031,Sv,1200,Sv 1200,...,Michèle Grieder,Riggisberg,Abegg-Stiftung,2022,NaN,China|Tod,g|s,BV002789766,Monographien der Abegg-Stiftung,außereuropäische Länder Einzelne textile Stoff...


In [83]:
# Export

export_columns = ['bvb', 'title_x', 'signature', 'inventory_nr','inventory_date', 'subject_list', 'lang', 'year_y']

merged[export_columns].to_csv('data/csv/freihand_signatures.csv', index=False)